In [54]:
import os
import pandas as pd

from pathlib import Path


readmissionTestDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240614', 'data', 'wb_365_wa_3', 'splits_v1', 'readmission', 'readmission_normal_test.csv'), sep='\t')
readmissionTrainDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240614', 'data', 'wb_365_wa_3', 'splits_v1', 'readmission', 'readmission_normal_train.csv'), sep='\t')
readmissionValidateDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240614', 'data', 'wb_365_wa_3', 'splits_v1', 'readmission', 'readmission_normal_validate.csv'), sep='\t')
readmissionDf = pd.concat([readmissionTestDf, readmissionTrainDf, readmissionValidateDf], ignore_index=True)
admissionsDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240531', 'etc', '2023-11-10-blood-admissions.csv'))
journeySuperEpisodeDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240531', 'etc', 'SuperBugSuperEpisode-72hr-2024-03-27.csv'))
journeyMasterDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240531', 'etc', 'SuperBugJourney-72hr-2024-03-27.csv'))

readmissionTestDf.shape, readmissionTrainDf.shape, readmissionValidateDf.shape, readmissionDf.shape, admissionsDf.shape, journeySuperEpisodeDf.shape, journeyMasterDf.shape

/tmp/ipykernel_7368/3223134191.py:12: DtypeWarning: Columns (10,17,26,31,37,44) have mixed types. Specify dtype option on import or set low_memory=False.
  journeySuperEpisodeDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240531', 'etc', 'SuperBugSuperEpisode-72hr-2024-03-27.csv'))
/tmp/ipykernel_7368/3223134191.py:13: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  journeyMasterDf = pd.read_csv(Path(os.environ['EHR_DATA_BASE'], 'blood_pos_cohort_20240531', 'etc', 'SuperBugJourney-72hr-2024-03-27.csv'))


((3781, 3),
 (29979, 3),
 (1848, 3),
 (35608, 3),
 (436214, 20),
 (95491, 46),
 (46258, 16))

In [55]:
# mergedDf = readmissionDf.merge(
#         admissionsDf,
#         how='inner',
#         left_on=['person_id', 'visit_occurrence_id'],
#         right_on=['PATIENT_ID', 'EPISODE_ID']
#     ).merge(
#         journeyDf[['PATIENT_ID', 'JOURNEY_ID', 'EPISODE_ID']],
#         how='inner',
#         on=['PATIENT_ID', 'EPISODE_ID']
#     )[['PATIENT_ID', 'EPISODE_ID', 'JOURNEY_ID', 'start_date', 'end_date']].drop_duplicates()
# mergedDf

# mergedDf = admissionsDf.merge(
#     journeyDf[['PATIENT_ID', 'JOURNEY_ID', 'EPISODE_ID']],
#     how='inner',
#     on=['PATIENT_ID', 'EPISODE_ID']
# ).merge(
#     readmissionDf,
#     how='inner',
#     left_on=['PATIENT_ID', 'JOURNEY_ID'],
#     right_on=['person_id', 'visit_occurrence_id']
# )
# # [['PATIENT_ID', 'EPISODE_ID', 'JOURNEY_ID', 'start_date', 'end_date']].drop_duplicates()
# mergedDf[['person_id', 'visit_occurrence_id']].drop_duplicates()

mergedDf = readmissionDf.merge(
    admissionsDf,
    how='inner',
    left_on=['person_id', 'visit_occurrence_id'],
    right_on=['PATIENT_ID', 'EPISODE_ID']
).drop(
    columns=['person_id', 'visit_occurrence_id']
).merge(
    journeySuperEpisodeDf[['PATIENT_ID', 'JOURNEY_ID', 'EPISODE_ID']],
    how='inner',
    on=['PATIENT_ID', 'EPISODE_ID']
).merge(
    journeyMasterDf[['PATIENT_ID', 'JOURNEY_ID', 'JOURNEY_START_DATETIME', 'JOURNEY_END_DATETIME']].drop_duplicates(),
    how='inner',
    on=['PATIENT_ID', 'JOURNEY_ID']
)
mergedDf[['PATIENT_ID', 'EPISODE_ID']].drop_duplicates().shape, readmissionDf[['person_id', 'visit_occurrence_id']].drop_duplicates().shape

((7307, 2), (7769, 2))

In [63]:
mergedDf.PATIENT_ID.value_counts().reset_index()[:20]

,PATIENT_ID,count
0,2084373,748
1,2019594,510
2,2164014,495
3,2128945,494
4,2130874,408
5,1780705,400
6,2371602,388
7,2167758,378
8,2594489,372
9,2094287,360


In [64]:
mergedDf[(mergedDf.PATIENT_ID == 2130874)][['PATIENT_ID', 'EPISODE_ID', 'JOURNEY_ID', 'start_date', 'end_date', 'JOURNEY_START_DATETIME', 'JOURNEY_END_DATETIME', 'READMISSION']].drop_duplicates().sort_values(by=['start_date', 'end_date'])

,PATIENT_ID,EPISODE_ID,JOURNEY_ID,start_date,end_date,JOURNEY_START_DATETIME,JOURNEY_END_DATETIME,READMISSION
11747,2130874,9779053,9779053,2018-12-06 21:55:33.000,2018-12-06 22:58:00.000,12/6/2018 21:55,1/5/2019 12:20,1
12064,2130874,631655,631655,2019-01-11 09:16:00.000,2019-01-11 12:38:00.000,1/11/2019 9:16,1/11/2019 12:38,0
11753,2130874,633081,633081,2019-01-14 16:55:00.000,2019-02-08 12:00:00.000,1/14/2019 16:55,2/8/2019 12:00,0
12144,2130874,644514,644514,2019-02-15 09:17:00.000,2019-02-15 13:25:00.000,2/15/2019 9:17,2/15/2019 13:25,0
11843,2130874,650325,650325,2019-03-01 16:28:00.000,2019-03-08 12:04:00.000,3/1/2019 16:28,3/11/2019 15:40,0
11915,2130874,653527,650325,2019-03-11 09:25:00.000,2019-03-11 15:40:00.000,3/1/2019 16:28,3/11/2019 15:40,0
11966,2130874,655465,655465,2019-03-15 08:36:00.000,2019-03-26 12:00:00.000,3/15/2019 8:36,3/28/2019 12:37,0
